In [1]:
print("hello world")
import pandas as pd
import evaluate
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, TFAutoModelForSeq2SeqLM,  AdamWeightDecay, TFT5ForConditionalGeneration

hello world


C:\Users\saira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", '3.0.0', split='train')
dataset_subset = dataset.select(list(range(30000)))

In [3]:
dataset

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [4]:
dataset_subset

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 30000
})

In [74]:
longest_article_length = max(len(example["article"]) for example in dataset)
longest_article_length

15925

In [75]:
shortest_article_length = min(len(example["article"]) for example in dataset)
shortest_article_length

48

In [76]:
longest_summary_length = max(len(example["highlights"]) for example in dataset)
longest_summary_length

7388

In [77]:
shortest_summary_length = min(len(example["highlights"]) for example in dataset)
shortest_summary_length

14

In [5]:
datasettest = load_dataset("cnn_dailymail", '3.0.0', split='train')


In [6]:
datasettest

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [7]:
datasettest

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [8]:
targetlist = datasettest['highlights'] 
sourcelist = datasettest['article']

In [9]:
len(targetlist[0:10000])

10000

In [10]:
train_df = pd.DataFrame(columns=['target', 'source'])

In [11]:
train_df['target'] = targetlist[0:10000]
train_df['source'] = sourcelist[0:10000]

In [12]:
train_df.count()

target    10000
source    10000
dtype: int64

In [13]:
train_df.head()

,target,source
0,Harry Potter star Daniel Radcliffe gets £20M f...,"LONDON, England (Reuters) -- Harry Potter star..."
1,Mentally ill inmates in Miami are housed on th...,Editor's note: In our Behind the Scenes series...
2,"NEW: ""I thought I was going to die,"" driver sa...","MINNEAPOLIS, Minnesota (CNN) -- Drivers who we..."
3,"Five small polyps found during procedure; ""non...",WASHINGTON (CNN) -- Doctors removed five small...
4,"NEW: NFL chief, Atlanta Falcons owner critical...",(CNN) -- The National Football League has ind...


In [14]:
subset = datasettest.select(list(range(10000)))

In [15]:
len(dataset['highlights'])

287113

In [16]:
for item in dataset['article']:
    print('summarize: '+item)
    break

summarize: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Det

In [17]:
dataset.num_rows

287113

In [18]:
dataset_subset = dataset_subset.train_test_split(test_size=0.2)

In [19]:
dataset_subset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 24000
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 6000
    })
})

In [4]:
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [21]:
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["highlights"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [22]:
tokenized_data = dataset_subset.map(preprocess_function, batched=True)

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map: 100%|██████████| 6000/6000 [00:12<00:00, 467.33 examples/s]


In [23]:
for doc in train_df['source']:
    print("summarize: "+doc)
    break

summarize: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Det

In [24]:
train_df.head()

,target,source
0,Harry Potter star Daniel Radcliffe gets £20M f...,"LONDON, England (Reuters) -- Harry Potter star..."
1,Mentally ill inmates in Miami are housed on th...,Editor's note: In our Behind the Scenes series...
2,"NEW: ""I thought I was going to die,"" driver sa...","MINNEAPOLIS, Minnesota (CNN) -- Drivers who we..."
3,"Five small polyps found during procedure; ""non...",WASHINGTON (CNN) -- Doctors removed five small...
4,"NEW: NFL chief, Atlanta Falcons owner critical...",(CNN) -- The National Football League has ind...


In [25]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")
rouge = evaluate.load("rouge")

In [26]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [27]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [28]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_data["train"],
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_data["test"],
    shuffle=False,
    batch_size=4,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [29]:
model.compile(optimizer=optimizer)

In [30]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=1)

6000/6000 [==============================] - 30135s 5s/step - loss: 1.9271 - val_loss: 1.6882


In [31]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
# model.save_weights('NLP_project/checkpoints/my_checkpoint') 

In [33]:
#model.save('NLP_project/checkpoints/t5summarizer.keras')

In [34]:
#model.save('NLP_project/checkpoints/t5summarizerr')

In [35]:
model.save_pretrained('/t5summarizerrrr')

In [2]:
from transformers import TFT5ForConditionalGeneration

nmodel = TFT5ForConditionalGeneration.from_pretrained('/t5summarizerrrr')

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /t5summarizerrrr.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [3]:
nmodel.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
dataset_subset

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 30000
})

In [8]:
dataset_subset['test']['id'] == '1f9aed77465301d7f3787a29555754a03728c4b7'

KeyError: "Column test not in the dataset. Current columns in the dataset: ['article', 'highlights', 'id']"

In [40]:
testdict = dataset_subset['test'][0]

In [51]:
testdict['article'] = "summarize: "+testdict['article']
testdict['article']

'summarize: summarize: New York (CNN) -- A mix of protesters gathered again Friday in cities across the country, decrying a loosely defined list of financial problems and mixing in places with others marking the 10-year anniversary of the U.S.-led invasion of Afghanistan. Demonstrators in New York and Washington appeared to congregate over both the Afghan conflict, arguably America\'s longest war, and in protest against the widening disparities between rich and poor and corporate greed, among other grievances. New York Police Commissioner Ray Kelly said an investigation is under way after protesters claimed officers used excessive force when corralling demonstrators earlier this week. He also noted that demonstrations had cost tax payers $1.9 million in overtime costs for the city\'s law enforcement. CNN affiliate stations also broadcast images of crowds that gathered in Austin, Texas, as well as Minneapolis, Minnesota; Seattle, Washington and Atlanta, Georgia. The activity came a day 

In [42]:
testdict['highlights']

'NEW: NY police commissioner: Police overtime has cost $1.9 million so far .\nWall Street protests blend with war demonstrations in New York and D.C.\nCrowds gather Austin, Minneapolis, Seattle and Atlanta .\nFriday marked the 21st day of the grass-roots protests .'

In [10]:
text = 'summarize: New York (CNN) -- A mix of protesters gathered again Friday in cities across the country, decrying a loosely defined list of financial problems and mixing in places with others marking the 10-year anniversary of the U.S.-led invasion of Afghanistan. Demonstrators in New York and Washington appeared to congregate over both the Afghan conflict, arguably America\'s longest war, and in protest against the widening disparities between rich and poor and corporate greed, among other grievances. New York Police Commissioner Ray Kelly said an investigation is under way after protesters claimed officers used excessive force when corralling demonstrators earlier this week. He also noted that demonstrations had cost tax payers $1.9 million in overtime costs for the city\'s law enforcement. CNN affiliate stations also broadcast images of crowds that gathered in Austin, Texas, as well as Minneapolis, Minnesota; Seattle, Washington and Atlanta, Georgia. The activity came a day after President Barack Obama discussed the growing movement, saying demonstrators "are giving voice to a more broad-based frustration about how our financial system works." Speaking at a White House news conference, Obama also defended the country\'s financial sector, which has taken the brunt of protesters\' criticism, focusing on Wall Street and its regulators\' purported role in expanding economic disparities. "We have to have a strong, effective financial sector in order for us to grow," the president said. Still, Obama discussed a need to pursue action aimed at improving government oversight and blamed Republican lawmakers for obstructing financial reforms. Friday marked the 21st day of the grass-roots Wall Street protests. Demonstrations have erupted in more than a dozen cities throughout the week, ranging from thousands who marched in lower Manhattan Wednesday after receiving support from local unions, to the dozens of college students who staged walkouts at various college campuses. Open Story: See iReports from the protests. The movement started in New York and some of the protests there have been marred by scuffles with police. New York authorities set up at least one vehicle checkpoint as police appeared in larger numbers throughout the financial district Thursday and established a perimeter around Zuccotti Park, which is considered a rallying point for the largely leaderless movement in that city. "We hope that our message continues to resonate with everyone who has felt disenfranchised by the current state of our country," said Tyler Combelic, a spokesman for the Occupy Wall Street group. He said they plan to "continue the protest until the message reaches every house in the United States." The specifics of that message remains largely unclear. CNN\'s Susan Candiotti, Carey Bodenheimer, Ross Levitt and Jason Kessler contributed to this report.'

In [11]:
basesum = 'NEW: NY police commissioner: Police overtime has cost $1.9 million so far .\nWall Street protests blend with war demonstrations in New York and D.C.\nCrowds gather Austin, Minneapolis, Seattle and Atlanta .\nFriday marked the 21st day of the grass-roots protests .'

In [12]:
inputs = tokenizer(text, return_tensors="tf").input_ids
outputs = nmodel.generate(inputs, max_new_tokens=100, do_sample=False)
t = tokenizer.decode(outputs[0], skip_special_tokens=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (620 > 512). Running this sequence through the model will result in indexing errors


In [13]:
print(t)

NEW: President Obama says protesters are giving voice to "a more broad-based frustration" NEW: The protests have erupted in more than a dozen cities throughout the week. NEW: The protests have been marred by scuffles with police. NEW: The protests are marking the 10-year anniversary of the U.S.-led invasion of Afghanistan.


In [45]:
import tensorflow as tf

In [46]:
#new_model = tf.keras.models.load_model('NLP_project/checkpoints/t5summarizer.keras')
#new_model = TFT5ForConditionalGeneration.from_pretrained('NLP_project/checkpoints/t5summarizerr')

In [14]:
from nltk.translate.bleu_score import sentence_bleu
import nltk

In [15]:
reference_summary_tokens = nltk.word_tokenize(basesum)
generated_summary_tokens = nltk.word_tokenize(t)

#     # Calculate BLEU score
bleu_score = sentence_bleu([reference_summary_tokens], generated_summary_tokens)
print(bleu_score)

4.4484724783464415e-155


C:\Users\saira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\saira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [49]:
testdict['highlights']

'NEW: NY police commissioner: Police overtime has cost $1.9 million so far .\nWall Street protests blend with war demonstrations in New York and D.C.\nCrowds gather Austin, Minneapolis, Seattle and Atlanta .\nFriday marked the 21st day of the grass-roots protests .'

In [50]:
testdict['article'] 

'summarize: New York (CNN) -- A mix of protesters gathered again Friday in cities across the country, decrying a loosely defined list of financial problems and mixing in places with others marking the 10-year anniversary of the U.S.-led invasion of Afghanistan. Demonstrators in New York and Washington appeared to congregate over both the Afghan conflict, arguably America\'s longest war, and in protest against the widening disparities between rich and poor and corporate greed, among other grievances. New York Police Commissioner Ray Kelly said an investigation is under way after protesters claimed officers used excessive force when corralling demonstrators earlier this week. He also noted that demonstrations had cost tax payers $1.9 million in overtime costs for the city\'s law enforcement. CNN affiliate stations also broadcast images of crowds that gathered in Austin, Texas, as well as Minneapolis, Minnesota; Seattle, Washington and Atlanta, Georgia. The activity came a day after Presi

In [55]:
from rouge import Rouge

In [56]:
rouge = Rouge()

In [57]:
print(rouge.get_scores(t, testdict['highlights']))

[{'rouge-1': {'r': 0.1794871794871795, 'p': 0.175, 'f': 0.17721518487421903}, 'rouge-2': {'r': 0.024390243902439025, 'p': 0.02127659574468085, 'f': 0.02272726775051762}, 'rouge-l': {'r': 0.1794871794871795, 'p': 0.175, 'f': 0.17721518487421903}}]


In [65]:
dataset

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})